# Assignment

## Data Fetching

In [3]:
import requests
from pymongo import MongoClient, errors

# MongoDB Configuration
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "host_data"
CROWDSTRIKE_COLLECTION = "crowdstrike_hosts"
QUALYS_COLLECTION = "qualys_hosts"

# API Configuration
headers = {
    'accept': 'application/json',
    'token': 'armis-login@armis.com_60974105-5053-4267-b16e-392e8165c89a',
    'content-type': 'application/x-www-form-urlencoded',
}

# API Endpoints
crowdstrike_url = 'https://api.recruiting.app.silk.security/api/crowdstrike/hosts/get'
qualys_url = 'https://api.recruiting.app.silk.security/api/qualys/hosts/get'

# Pagination parameters
batch_size = 1

# MongoDB Client
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

# Ensure unique indexes
db[CROWDSTRIKE_COLLECTION].create_index("hostname", unique=True)
db[QUALYS_COLLECTION].create_index("name", unique=True)

def fetch_and_store_data(api_url, collection_name, unique_key):
    skip = 0  # Start with the first batch

    while True:
        # Set pagination parameters
        params = {
            'skip': skip,
            'limit': batch_size,
        }

        # Fetch data from the API
        response = requests.post(api_url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching data from {api_url}: {response.status_code} - {response.text}")
            break
        
        data = response.json()
        
        # If no data is returned, we've fetched everything
        if not data:
            print(f"No more data to fetch from {api_url}.")
            break

        # Insert into MongoDB
        for item in data:
            try:
                db[collection_name].insert_one(item)
                print(f"Inserted into {collection_name}: {item[unique_key]}")
            except errors.DuplicateKeyError:
                print(f"Duplicate skipped in {collection_name}: {item[unique_key]}")
            except Exception as e:
                print(f"Error inserting into {collection_name}: {e}")
        
        # Increment the skip to fetch the next batch
        skip += batch_size

# Fetch and store data for both APIs
print("Fetching and storing Crowdstrike data...")
fetch_and_store_data(crowdstrike_url, CROWDSTRIKE_COLLECTION, "hostname")

print("Fetching and storing Qualys data...")
fetch_and_store_data(qualys_url, QUALYS_COLLECTION, "name")

print("Data fetching and storage completed.")


Fetching and storing Crowdstrike data...
Inserted into crowdstrike_hosts: ip-172-31-93-76.ec2.internal
Inserted into crowdstrike_hosts: ip-172-31-14-41.ec2.internal
Inserted into crowdstrike_hosts: ip-172-31-19-223.ec2.internal
Inserted into crowdstrike_hosts: ip-172-31-94-229.ec2.internal
Inserted into crowdstrike_hosts: ip-172-31-27-66.ec2.internal
Inserted into crowdstrike_hosts: ip-172-31-60-172.ec2.internal
Inserted into crowdstrike_hosts: yoavs-mbp-2.lan
Error fetching data from https://api.recruiting.app.silk.security/api/crowdstrike/hosts/get: 500 - Error invalid skip/limit combo (>number of hosts)
Fetching and storing Qualys data...
Inserted into qualys_hosts: ip-172-31-19-223.ec2.internal
Inserted into qualys_hosts: ip-172-31-93-76.ec2.internal
Inserted into qualys_hosts: ip-172-31-14-41.ec2.internal
Inserted into qualys_hosts: ip-172-31-27-66.ec2.internal
Inserted into qualys_hosts: ip-172-31-94-229.ec2.internal
Inserted into qualys_hosts: EC2AMAZ-E8FAE79
Inserted into qualy

## Checking the data in mongodb

In [ ]:
# from pymongo import MongoClient

# # Connect to MongoDB
# client = MongoClient("mongodb://localhost:27017/")
# db = client["host_data"]

# # Count and print the number of documents in each collection
# qualys_count = db["qualys_hosts"].count_documents({})
# crowdstrike_count = db["crowdstrike_hosts"].count_documents({})

# print(f"Number of Qualys Hosts: {qualys_count}")
# print(f"Number of Crowdstrike Hosts: {crowdstrike_count}")

# # Print Qualys Hosts
# print("\nQualys Hosts:")
# for host in db["qualys_hosts"].find():
#     print(host)

# # Print Crowdstrike Hosts
# print("\nCrowdstrike Hosts:")
# for host in db["crowdstrike_hosts"].find():
#     print(host)


Number of Qualys Hosts: 7
Number of Crowdstrike Hosts: 7

Qualys Hosts:
{'_id': 305003660, 'account': {'list': [{'HostAssetAccount': {'username': 'ssm-user'}}, {'HostAssetAccount': {'username': 'ec2-user'}}, {'HostAssetAccount': {'username': 'root'}}]}, 'address': '172.31.19.223', 'agentInfo': {'location': 'Ashburn,Virginia United States', 'locationGeoLatitude': '39.0469', 'lastCheckedIn': {'$date': '2023-07-26T04:27:35.000Z'}, 'locationGeoLongtitude': '-77.4903', 'agentVersion': '6.0.0.41', 'manifestVersion': {'sca': 'VULNSIGS-SCA-2.5.824.2-1', 'vm': 'VULNSIGS-VM-2.5.825.2-1'}, 'activatedModule': 'AGENT_VM,AGENT_SCA,AGENT_PM', 'activationKey': {'title': 'Cloud agent key', 'activationId': '79535c40-6b10-4fe8-bec6-6f828736b5bf'}, 'agentConfiguration': {'id': 1000, 'name': 'Initial Profile'}, 'status': 'STATUS_ACTIVE', 'chirpStatus': 'Inventory Manifest Downloaded', 'connectedFrom': '18.212.216.193', 'agentId': '1ecc8a88-5912-4058-b731-94a04a48db11', 'platform': 'Linux'}, 'biosDescriptio

## Delete all the data from mongodb

In [ ]:
# from pymongo import MongoClient

# # MongoDB Configuration
# MONGO_URI = "mongodb://localhost:27017/"
# DB_NAME = "host_data"

# # MongoDB Client
# client = MongoClient(MONGO_URI)

# def remove_database():
#     """
#     Remove the entire database.
#     """
#     client.drop_database(DB_NAME)
#     print(f"Database '{DB_NAME}' and all its collections have been removed.")

# if __name__ == "__main__":
#     remove_database()


Database 'host_data' and all its collections have been removed.


## Normalization

In [90]:
import pandas as pd
from pymongo import MongoClient

# MongoDB Configuration
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "host_data"
QUALYS_COLLECTION = "qualys_hosts"
CROWDSTRIKE_COLLECTION = "crowdstrike_hosts"

# MongoDB Client
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

# Fetch data from Qualys collection
qualys_data = list(db[QUALYS_COLLECTION].find())
qualys_df = pd.DataFrame(qualys_data)

# Fetch data from Crowdstrike collection
crowdstrike_data = list(db[CROWDSTRIKE_COLLECTION].find())
crowdstrike_df = pd.DataFrame(crowdstrike_data)

print("Qualys DataFrame:")
print(qualys_df.head())

print("\nCrowdstrike DataFrame:")
print(crowdstrike_df.head())

Qualys DataFrame:
         _id                                            account  \
0  305003660  {'list': [{'HostAssetAccount': {'username': 's...   
1  305004025  {'list': [{'HostAssetAccount': {'username': 'e...   
2  305005279  {'list': [{'HostAssetAccount': {'username': 's...   
3  305005680  {'list': [{'HostAssetAccount': {'username': 's...   
4  305005702  {'list': [{'HostAssetAccount': {'username': 'e...   

         address                                          agentInfo  \
0  172.31.19.223  {'location': 'Ashburn,Virginia United States',...   
1   172.31.93.76  {'location': 'Ashburn,Virginia United States',...   
2   172.31.14.41  {'location': 'Ashburn,Virginia United States',...   
3   172.31.27.66  {'location': 'Ashburn,Virginia United States',...   
4  172.31.94.229  {'location': 'Ashburn,Virginia United States',...   

              biosDescription cloudProvider               created  \
0   Xen 4.2.amazon 08/24/2006           AWS  2022-08-18T22:18:40Z   
1  Xen 4.11.am

In [143]:
import pandas as pd

def flatten_dict(nested_dict, parent_key='', sep='_'):
    """
    Recursively flattens a nested dictionary or list into a flat dictionary.
    """
    items = []
    for k, v in nested_dict.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for idx, item in enumerate(v):
                items.extend(flatten_dict({f"{new_key}_{idx}": item}).items())
        else:
            items.append((new_key, v))
    return dict(items)

def normalize_to_single_table(dataset):
    """
    Flattens and normalizes deeply nested data into a single DataFrame.
    
    Args:
        dataset (list): List of dictionaries representing the dataset.
    
    Returns:
        pd.DataFrame: A single normalized DataFrame.
    """
    # Flatten each record in the dataset
    normalized_data = [flatten_dict(record) for record in dataset]
    
    # Convert the list of flattened dictionaries into a DataFrame
    normalized_df = pd.DataFrame(normalized_data)
    return normalized_df


In [144]:
normalize_crowdstrike_df = normalize_to_single_table(crowdstrike_data)

In [145]:
normalize_qualys_df = normalize_to_single_table(qualys_data)

In [148]:
# Join the dataframes
merged_df = pd.merge(normalize_qualys_df, normalize_crowdstrike_df, left_on='name', right_on='hostname', suffixes=('_qualys', '_crowdstrike'))

# Display the merged dataframe
merged_df

_id_qualys account_list_0_HostAssetAccount_username  \
0   305003660                                 ssm-user   
1   305004025                                 ec2-user   
2   305005279                                 ssm-user   
3   305005680                                 ssm-user   
4   305005702                                 ec2-user   
5   305019294                                 ec2-user   

  account_list_1_HostAssetAccount_username  \
0                                 ec2-user   
1                                     root   
2                                 ec2-user   
3                                 ec2-user   
4                                     root   
5                                     root   

  account_list_2_HostAssetAccount_username        address  \
0                                     root  172.31.19.223   
1                                      NaN   172.31.93.76   
2                                     root   172.31.14.41   
3                                     root   172.31.27.66   
4                                      NaN  172.31.94.229   
5                                      NaN  172.31.60.172   

               agentInfo_location agentInfo_locationGeoLatitude  \
0  Ashburn,Virginia United States                       39.0469   
1  Ashburn,Virginia United States                       39.0469   
2  Ashburn,Virginia United States                       39.0469   
3  Ashburn,Virginia United States                       39.0469   
4  Ashburn,Virginia United States                       39.0469   
5  Ashburn,Virginia United States                       39.0469   

  agentInfo_lastCheckedIn_$date agentInfo_locationGeoLongtitude  \
0      2023-07-26T04:27:35.000Z                        -77.4903   
1      2022-08-31T10:26:24.000Z                        -77.4903   
2      2023-02-07T20:27:13.000Z                        -77.4903   
3      2023-07-19T10:43:57.000Z                        -77.4903   
4      2022-08-31T10:26:44.000Z                        -77.4903   
5      2022-08-31T10:26:30.000Z                        -77.4903   

  agentInfo_agentVersion agentInfo_manifestVersion_sca  \
0               6.0.0.41      VULNSIGS-SCA-2.5.824.2-1   
1               5.0.0.31      VULNSIGS-SCA-2.5.569.2-1   
2               5.6.0.29      VULNSIGS-SCA-2.5.694.2-1   
3               6.0.0.41      VULNSIGS-SCA-2.5.818.3-2   
4               5.0.0.31      VULNSIGS-SCA-2.5.569.2-1   
5               5.0.0.31      VULNSIGS-SCA-2.5.569.2-1   

  agentInfo_manifestVersion_vm    agentInfo_activatedModule  \
0      VULNSIGS-VM-2.5.825.2-1  AGENT_VM,AGENT_SCA,AGENT_PM   
1      VULNSIGS-VM-2.5.569.2-1  AGENT_VM,AGENT_PM,AGENT_SCA   
2      VULNSIGS-VM-2.5.694.2-1  AGENT_PM,AGENT_VM,AGENT_SCA   
3      VULNSIGS-VM-2.5.818.3-2  AGENT_VM,AGENT_SCA,AGENT_PM   
4      VULNSIGS-VM-2.5.569.2-1  AGENT_VM,AGENT_SCA,AGENT_PM   
5      VULNSIGS-VM-2.5.569.2-1  AGENT_SCA,AGENT_VM,AGENT_PM   

  agentInfo_activationKey_title  agentInfo_activationKey_activationId  \
0               Cloud agent key  79535c40-6b10-4fe8-bec6-6f828736b5bf   
1               Cloud agent key  79535c40-6b10-4fe8-bec6-6f828736b5bf   
2               Cloud agent key  79535c40-6b10-4fe8-bec6-6f828736b5bf   
3               Cloud agent key  79535c40-6b10-4fe8-bec6-6f828736b5bf   
4               Cloud agent key  79535c40-6b10-4fe8-bec6-6f828736b5bf   
5               Cloud agent key  79535c40-6b10-4fe8-bec6-6f828736b5bf   

   agentInfo_agentConfiguration_id agentInfo_agentConfiguration_name  \
0                             1000                   Initial Profile   
1                             1000                   Initial Profile   
2                             1000                   Initial Profile   
3                             1000                   Initial Profile   
4                             1000                   Initial Profile   
5                             1000                   Initial Profile   

  agentInfo_status          agentInfo_c

In [158]:
# Selected keys for visualization
selected_keys = [
    '_id_qualys', 'address', 'agentInfo_location', 'agentInfo_locationGeoLatitude',
    'agentInfo_locationGeoLongtitude', 'agentInfo_agentVersion', 'agentInfo_status',
    'biosDescription', 'cloudProvider', 'dnsHostName', 'fqdn', 'isDockerHost',
    'manufacturer', 'model', 'name', 'os', 'processor_list_0_HostAssetProcessor_name',
    'processor_list_0_HostAssetProcessor_speed', 'software_list_0_HostAssetSoftware_name',
    'software_list_0_HostAssetSoftware_version', 'totalMemory',
    'volume_list_0_HostAssetVolume_name', 'volume_list_0_HostAssetVolume_size',
    'vuln_list_0_HostAssetVuln_qid', 'vuln_list_0_HostAssetVuln_lastFound'
]

final_normal_df = merged_df[selected_keys]

## Store it in CSV

In [159]:
final_normal_df.to_csv('final_normal_df.csv', index=False)

In [160]:
final_normal_df

,_id_qualys,address,agentInfo_location,agentInfo_locationGeoLatitude,agentInfo_locationGeoLongtitude,agentInfo_agentVersion,agentInfo_status,biosDescription,cloudProvider,dnsHostName,fqdn,isDockerHost,manufacturer,model,name,os,processor_list_0_HostAssetProcessor_name,processor_list_0_HostAssetProcessor_speed,software_list_0_HostAssetSoftware_name,software_list_0_HostAssetSoftware_version,totalMemory,volume_list_0_HostAssetVolume_name,volume_list_0_HostAssetVolume_size,vuln_list_0_HostAssetVuln_qid,vuln_list_0_HostAssetVuln_lastFound
0,305003660,172.31.19.223,"Ashburn,Virginia United States",39.0469,-77.4903,6.0.0.41,STATUS_ACTIVE,Xen 4.2.amazon 08/24/2006,AWS,ip-172-31-19-223.ec2.internal,ip-172-31-19-223.ec2.internal,false,Xen,HVM domU,ip-172-31-19-223.ec2.internal,Amazon Linux 2,Intel(R) Xeon(R) CPU E5-2676 v3 @ 2.40GHz,2400,SentinelAgent,23.1.2.9-1.x86_64,966,/dev,4.969595e+08,354794,2023-07-26T04:20:52Z
1,305004025,172.31.93.76,"Ashburn,Virginia United States",39.0469,-77.4903,5.0.0.31,STATUS_INACTIVE,Xen 4.11.amazon 08/24/2006,AWS,ip-172-31-93-76.ec2.internal,ip-172-31-93-76.ec2.internal,false,Xen,HVM domU,ip-172-31-93-76.ec2.internal,Amazon Linux 2,Intel(R) Xeon(R),2400,thunderbird,91.11.0-2.amzn2.0.1.x86_64,966,tmpfs,5.062083e+08,105098,2022-08-31T04:36:17Z
2,305005279,172.31.14.41,"Ashburn,Virginia United States",39.0469,-77.4903,5.6.0.29,STATUS_INACTIVE,Xen 4.2.amazon 08/24/2006,AWS,ip-172-31-14-41.ec2.internal,ip-172-31-14-41.ec2.internal,false,Xen,HVM domU,ip-172-31-14-41.ec2.internal,Amazon Linux 2,Intel(R) Xeon(R),2300,amazon-ssm-agent,3.2.532.0-1.x86_64,1971,/,NaN,354023,2023-02-07T20:06:39Z
3,305005680,172.31.27.66,"Ashburn,Virginia United States",39.0469,-77.4903,6.0.0.41,STATUS_INACTIVE,Xen 4.2.amazon 08/24/2006,AWS,ip-172-31-27-66.ec2.internal,ip-172-31-27-66.ec2.internal,false,Xen,HVM domU,ip-172-31-27-66.ec2.internal,Amazon Linux 2,Intel(R) Xeon(R) CPU E5-2676 v3 @ 2.40GHz,2400,OpenSSL,1.0.2k-fips,966,/dev,4.969841e+08,378451,2023-07-19T10:04:28Z
4,305005702,172.31.94.229,"Ashburn,Virginia United States",39.0469,-77.4903,5.0.0.31,STATUS_INACTIVE,Xen 4.11.amazon 08/24/2006,AWS,ip-172-31-94-229.ec2.internal,ip-172-31-94-229.ec2.internal,false,Xen,HVM domU,ip-172-31-94-229.ec2.internal,Amazon Linux 2,Intel(R) Xeon(R),2400,boost-date-time,1.53.0-27.amzn2.0.5.x86_64,966,tmpfs,5.062001e+08,370098,2022-08-31T04:53:20Z
5,305019294,172.31.60.172,"Ashburn,Virginia United States",39.0469,-77.4903,5.0.0.31,STATUS_INACTIVE,Xen 4.2.amazon 08/24/2006,AWS,ip-172-31-60-172.ec2.internal,ip-172-31-60-172.ec2.internal,false,Xen,HVM domU,ip-172-31-60-172.ec2.internal,Amazon Linux 2,Intel(R) Xeon(R),2300,amazon-ssm-agent,3.1.1732.0-1.x86_64,1972,tmpfs,1.033646e+09,354051,2022-08-31T03:53:24Z
